In [1]:
!gdown 1OS2Aurb2oKmooCeYwYw0cnp542t7dyHo
!gdown 1Oibb4sFzJNLt0iBWOD4LAIKaWT7cCNT1

Downloading...
From: https://drive.google.com/uc?id=1OS2Aurb2oKmooCeYwYw0cnp542t7dyHo
To: /content/test_data.csv
100% 565k/565k [00:00<00:00, 22.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Oibb4sFzJNLt0iBWOD4LAIKaWT7cCNT1
To: /content/train_data.csv
100% 2.24M/2.24M [00:00<00:00, 146MB/s]


In [2]:
import numpy as np
import pandas as pd
import gensim
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score
from xgboost import XGBClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

In [7]:
train_df = pd.read_csv("train_data.csv")
test_df = pd.read_csv("test_data.csv")

In [8]:
train_df = train_df[train_df['confidence'] >= 0.6].reset_index(drop=True)
train_df.shape

(2420, 111)

In [9]:
test_df = test_df[test_df['confidence'] >= 0.6].reset_index(drop=True)
test_df.shape

(613, 111)

In [ ]:
train_df = train_df[train_df['confidence'] >= 0.8].reset_index(drop=True)

In [ ]:
train_df.shape

(1832, 111)

In [ ]:
test_df = test_df[test_df['confidence'] >= 0.8].reset_index(drop=True)
test_df.shape

(465, 111)

In [ ]:
X_train = train_df['clean_text'].to_numpy()
y_train = train_df['target'].to_numpy()

In [ ]:
X_test = test_df['clean_text'].to_numpy()
y_test = test_df['target'].to_numpy()

# Word2Vec

In [ ]:
model_w2v = gensim.models.Word2Vec(window=15, min_count=2, vector_size=300)

In [ ]:
texts = []
for text in X_train:
  texts.append(text.split())
for text in X_test:
  texts.append(text.split())

In [ ]:
model_w2v.build_vocab(texts)

In [ ]:
model_w2v.train(texts, total_examples=model_w2v.corpus_count, epochs=50)

(3705239, 4372950)

In [ ]:
model_w2v.wv.most_similar("good")

[('tough', 0.5066089034080505),
 ('friend', 0.47309011220932007),
 ('bumble', 0.46908730268478394),
 ('supportive', 0.4679715931415558),
 ('distant', 0.4449012577533722),
 ('kind', 0.4368584156036377),
 ('caring', 0.4202074706554413),
 ('poodle', 0.41564926505088806),
 ('describe', 0.41392722725868225),
 ('entertain', 0.4101490080356598)]

In [ ]:
model_w2v.wv.doesnt_match(['good', 'great', 'nice', 'bad'])

'bad'

In [ ]:
model_w2v.save('word2vec.model')

In [ ]:
def document_vector(doc):
    voc = set(model_w2v.wv.index_to_key)
    doc = doc.split()
    doc = [word for word in doc if word in voc]
    return np.mean(model_w2v.wv[doc], axis=0)

In [ ]:
X_train_w2v = []
for review in X_train:
    X_train_w2v.append(document_vector(review))

In [ ]:
X_test_w2v = []
for review in X_test:
    X_test_w2v.append(document_vector(review))

In [ ]:
np.array(X_train_w2v).shape

(1832, 300)

# Training and Testing

In [ ]:
def training_and_prediction(training_model, X_train, X_test, y_train, y_test):
  model = training_model
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  f1score = f1_score(y_test, y_pred)
  # print("Accuracy =", accuracy)
  # print("Confusion Matrix =\n", confusion_matrix(y_test, y_pred))
  # print(classification_report(y_test, y_pred))
  return {'Accuracy' : accuracy, 'Precision': precision, 'Recall' : recall, 'F1-Score' : f1score}

In [ ]:
def get_report(y_test, y_pred):
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  f1score = f1_score(y_test, y_pred)
  return {'Accuracy' : accuracy, 'Precision': precision, 'Recall' : recall, 'F1-Score' : f1score}

## Naive Bayes

In [ ]:
scaler = MinMaxScaler()
training_and_prediction(MultinomialNB(alpha=10), scaler.fit_transform(X_train_w2v), scaler.fit_transform(X_test_w2v), y_train, y_test)

{'Accuracy': 0.7462365591397849,
 'Precision': 0.7364341085271318,
 'Recall': 0.7916666666666666,
 'F1-Score': 0.7630522088353414}

In [ ]:
training_and_prediction(GaussianNB(), X_train_w2v, X_test_w2v, y_train, y_test)

{'Accuracy': 0.7397849462365591,
 'Precision': 0.7351778656126482,
 'Recall': 0.775,
 'F1-Score': 0.7545638945233266}

In [ ]:
training_and_prediction(BernoulliNB(alpha=1), X_train_w2v, X_test_w2v, y_train, y_test)

{'Accuracy': 0.7505376344086021,
 'Precision': 0.756198347107438,
 'Recall': 0.7625,
 'F1-Score': 0.7593360995850622}

## SVM

In [ ]:
training_and_prediction(SVC(C=2), X_train_w2v, X_test_w2v, y_train, y_test)

{'Accuracy': 0.7720430107526882,
 'Precision': 0.7723577235772358,
 'Recall': 0.7916666666666666,
 'F1-Score': 0.7818930041152262}

## Random Forest

In [ ]:
training_and_prediction(RandomForestClassifier(), X_train_w2v, X_test_w2v, y_train, y_test)

{'Accuracy': 0.7569892473118279,
 'Precision': 0.7509881422924901,
 'Recall': 0.7916666666666666,
 'F1-Score': 0.7707910750507099}

## Logistic Regression

In [ ]:
training_and_prediction(LogisticRegression(max_iter=500, solver='saga'), X_train_w2v, X_test_w2v, y_train, y_test)

{'Accuracy': 0.7591397849462366,
 'Precision': 0.7622950819672131,
 'Recall': 0.775,
 'F1-Score': 0.768595041322314}

## Gradient Boosting

In [ ]:
training_and_prediction(XGBClassifier(), X_train_w2v, X_test_w2v, y_train, y_test)

{'Accuracy': 0.7548387096774194,
 'Precision': 0.75,
 'Recall': 0.7875,
 'F1-Score': 0.7682926829268292}

## Deep Learning Techniques with Word2vec

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Bidirectional, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Tokenize text data
tokenizer = Tokenizer(num_words=7000)
tokenizer.fit_on_texts(np.concatenate((X_train, X_test), axis=0))
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

In [ ]:
high = 0
for temp in X_train_sequences:
  high = max(high, len(temp))
high

141

In [ ]:
# Pad sequences to same length
max_len = 150
X_train_padded_sequences = pad_sequences(X_train_sequences, maxlen=max_len)
X_test_padded_sequences = pad_sequences(X_test_sequences, maxlen=max_len)

In [ ]:
# Create embedding layer
embedding_dim = 300
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, embedding_dim))
for word, i in tokenizer.word_index.items():
    if word in model_w2v.wv:
        embedding_matrix[i] = model_w2v.wv[word]
embedding_layer = Embedding(len(tokenizer.word_index) + 1, embedding_dim, weights=[embedding_matrix], trainable=False)

### ANN

In [ ]:
def ann_models(num_neurons_layer1, num_neurons_layer2=0, dropout_rate=0):
    model_dense = Sequential()
    model_dense.add(embedding_layer)
    model_dense.add(GlobalAveragePooling1D())
    model_dense.add(Dense(num_neurons_layer1, activation='relu'))
    model_dense.add(Dropout(dropout_rate))
    if num_neurons_layer2 > 0:
      model_dense.add(Dense(num_neurons_layer2, activation='relu'))
    model_dense.add(Dense(1, activation='sigmoid'))
    model_dense.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model_dense.fit(X_train_padded_sequences, y_train, epochs=20, batch_size=32, verbose=0, callbacks=[early_stop], validation_data=(X_test_padded_sequences, y_test))
    y_pred_dense = model_dense.predict(X_test_padded_sequences)
    y_pred_dense = (y_pred_dense > 0.5).astype(int)
    return [{'num_neurons_layer1': num_neurons_layer1, 'num_neurons_layer2': num_neurons_layer2, 'dropout_rate': dropout_rate}, get_report(y_test, y_pred_dense)]


In [ ]:
for num_neurons_layer1 in [128, 64]:
  for num_neurons_layer2 in [64, 32, 0]:
    for dropout_rate in [0.0, 0.1, 0.2, 0.3, 0.4]:
      print(ann_models(num_neurons_layer1, num_neurons_layer2, dropout_rate))

15/15 [==============================] - 0s 5ms/step
[{'num_neurons_layer1': 128, 'num_neurons_layer2': 64, 'dropout_rate': 0.0}, {'Accuracy': 0.7698924731182796, 'Precision': 0.7736625514403292, 'Recall': 0.7833333333333333, 'F1-Score': 0.7784679089026916}]
15/15 [==============================] - 0s 8ms/step
[{'num_neurons_layer1': 128, 'num_neurons_layer2': 64, 'dropout_rate': 0.1}, {'Accuracy': 0.7698924731182796, 'Precision': 0.7714285714285715, 'Recall': 0.7875, 'F1-Score': 0.7793814432989691}]
15/15 [==============================] - 0s 6ms/step
[{'num_neurons_layer1': 128, 'num_neurons_layer2': 64, 'dropout_rate': 0.2}, {'Accuracy': 0.7720430107526882, 'Precision': 0.7913043478260869, 'Recall': 0.7583333333333333, 'F1-Score': 0.7744680851063831}]
15/15 [==============================] - 0s 9ms/step
[{'num_neurons_layer1': 128, 'num_neurons_layer2': 64, 'dropout_rate': 0.3}, {'Accuracy': 0.7827956989247312, 'Precision': 0.7957446808510639, 'Recall': 0.7791666666666667, 'F1-Score

## LSTM

In [ ]:
def lstm_models(num_neurons_layer1, dropout_rate=0):
    model_lstm = Sequential()
    model_lstm.add(embedding_layer)
    model_lstm.add(LSTM(128, dropout=dropout_rate))
    model_lstm.add(Dense(1, activation='sigmoid'))
    model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    model_lstm.fit(X_train_padded_sequences, y_train, epochs=20, batch_size=32, verbose=0, callbacks=[early_stop], validation_data=(X_test_padded_sequences, y_test))
    y_pred_lstm = model_lstm.predict(X_test_padded_sequences)
    y_pred_lstm = (y_pred_lstm > 0.5).astype(int)
    return [{'num_neurons_layer1': num_neurons_layer1, 'dropout_rate': dropout_rate}, get_report(y_test, y_pred_lstm)]

In [ ]:
for num_neurons_layer1 in [128, 64]:
  for dropout_rate in [0.0, 0.1, 0.2, 0.3, 0.4]:
    print(lstm_models(num_neurons_layer1, dropout_rate))

15/15 [==============================] - 2s 115ms/step
[{'num_neurons_layer1': 128, 'dropout_rate': 0.0}, {'Accuracy': 0.7741935483870968, 'Precision': 0.7710843373493976, 'Recall': 0.8, 'F1-Score': 0.7852760736196319}]
15/15 [==============================] - 2s 91ms/step
[{'num_neurons_layer1': 128, 'dropout_rate': 0.1}, {'Accuracy': 0.7763440860215054, 'Precision': 0.7615384615384615, 'Recall': 0.825, 'F1-Score': 0.7919999999999999}]
15/15 [==============================] - 2s 88ms/step
[{'num_neurons_layer1': 128, 'dropout_rate': 0.2}, {'Accuracy': 0.7806451612903226, 'Precision': 0.7974137931034483, 'Recall': 0.7708333333333334, 'F1-Score': 0.7838983050847458}]
15/15 [==============================] - 2s 91ms/step
[{'num_neurons_layer1': 128, 'dropout_rate': 0.3}, {'Accuracy': 0.7526881720430108, 'Precision': 0.7431906614785992, 'Recall': 0.7958333333333333, 'F1-Score': 0.7686116700201207}]
15/15 [==============================] - 2s 92ms/step
[{'num_neurons_layer1': 128, 'dropout

### Bidirectional LSTM

In [ ]:
model_bilstm = Sequential()
model_bilstm.add(embedding_layer)
model_bilstm.add(SpatialDropout1D(0.2))
model_bilstm.add(Bidirectional(LSTM(128, return_sequences=False)))
model_bilstm.add(Dropout(0.2))
model_bilstm.add(Dense(1, activation='sigmoid'))
model_bilstm.summary()

Model: "sequential_42"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 300)         2455800   
                                                                 
 spatial_dropout1d (Spatial  (None, None, 300)         0         
 Dropout1D)                                                      
                                                                 
 bidirectional (Bidirection  (None, 256)               439296    
 al)                                                             
                                                                 
 dropout_32 (Dropout)        (None, 256)               0         
                                                                 
 dense_96 (Dense)            (None, 1)                 257       
                                                                 
Total params: 2895353 (11.04 MB)
Trainable params: 43

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_bilstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_bilstm.fit(X_train_padded_sequences, y_train, epochs=10, batch_size=32, validation_data=(X_test_padded_sequences, y_test), callbacks=[early_stop])

Epoch 1/10
58/58 [==============================] - 39s 567ms/step - loss: 0.4954 - accuracy: 0.7642 - val_loss: 0.4779 - val_accuracy: 0.7871
Epoch 2/10
58/58 [==============================] - 34s 587ms/step - loss: 0.3785 - accuracy: 0.8291 - val_loss: 0.4812 - val_accuracy: 0.7720
Epoch 3/10
58/58 [==============================] - 33s 569ms/step - loss: 0.3413 - accuracy: 0.8537 - val_loss: 0.5099 - val_accuracy: 0.7677
Epoch 4/10
58/58 [==============================] - 34s 587ms/step - loss: 0.2985 - accuracy: 0.8772 - val_loss: 0.5244 - val_accuracy: 0.7677


In [ ]:
y_pred_bilstm = model_bilstm.predict(X_test_padded_sequences)
y_pred_bilstm = (y_pred_bilstm > 0.5).astype(int)
get_report(y_test, y_pred_bilstm)

15/15 [==============================] - 3s 157ms/step


{'Accuracy': 0.7870967741935484,
 'Precision': 0.7901234567901234,
 'Recall': 0.8,
 'F1-Score': 0.7950310559006211}

### GRU

In [ ]:
model_gru = Sequential()
model_gru.add(embedding_layer)
model_gru.add(SpatialDropout1D(0.2))
model_gru.add(GRU(128, return_sequences = False))
model_gru.add(Dropout(0.2))
model_gru.add(Dense(1, activation = 'sigmoid'))
model_gru.summary()

Model: "sequential_43"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 300)         2455800   
                                                                 
 spatial_dropout1d_1 (Spati  (None, None, 300)         0         
 alDropout1D)                                                    
                                                                 
 gru (GRU)                   (None, 128)               165120    
                                                                 
 dropout_33 (Dropout)        (None, 128)               0         
                                                                 
 dense_97 (Dense)            (None, 1)                 129       
                                                                 
Total params: 2621049 (10.00 MB)
Trainable params: 165249 (645.50 KB)
Non-trainable params: 2455800 (9.37 MB)
_________

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_gru.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_gru.fit(X_train_padded_sequences, y_train, epochs=10, batch_size=32, validation_data=(X_test_padded_sequences, y_test), callbacks=[early_stop])

Epoch 1/10
58/58 [==============================] - 17s 255ms/step - loss: 0.5883 - accuracy: 0.6741 - val_loss: 0.6192 - val_accuracy: 0.7441
Epoch 2/10
58/58 [==============================] - 14s 250ms/step - loss: 0.4318 - accuracy: 0.8046 - val_loss: 0.5307 - val_accuracy: 0.7462
Epoch 3/10
58/58 [==============================] - 15s 257ms/step - loss: 0.3696 - accuracy: 0.8362 - val_loss: 0.5340 - val_accuracy: 0.7613
Epoch 4/10
58/58 [==============================] - 16s 279ms/step - loss: 0.3319 - accuracy: 0.8614 - val_loss: 0.5697 - val_accuracy: 0.7548
Epoch 5/10
58/58 [==============================] - 15s 251ms/step - loss: 0.3080 - accuracy: 0.8624 - val_loss: 0.5934 - val_accuracy: 0.7613


In [ ]:
y_pred_gru = model_gru.predict(X_test_padded_sequences)
y_pred_gru = (y_pred_gru > 0.5).astype(int)
get_report(y_test, y_pred_gru)

15/15 [==============================] - 2s 75ms/step


{'Accuracy': 0.7462365591397849,
 'Precision': 0.7629310344827587,
 'Recall': 0.7375,
 'F1-Score': 0.75}